In [1]:
import pandas as pd
from konlpy.tag import Okt

In [3]:
okt = Okt()

In [2]:
file_path = 'SentiWord_Dict.txt'

In [3]:
# 사전 데이터 불러오기
df = pd.read_csv('SentiWord_Dict.txt', delimiter='\t', names=['단어', '점수'], header=None)

In [4]:
# 한글 부분만 필요하기 때문에, 사전에서 한글필터링 - 정규표현식
df = df[df['단어'].str.match(r'^[가-힣]+$')]

In [5]:
df.head(10)

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0
50,가난뱅이,-2.0
51,가난살이,-2.0
52,가난살이하다,-2.0
53,가난설음,-2.0
54,가난에,-2.0


#### 주식이라는 테마에 맞는 긍정/부정 단어 사전에 추가

In [23]:
# 긍정 단어 리스트
positive_words= ['상승','호재','수익','매수','강세',
                 '성장','기대감','안정성','배당','흥행',
                 '신규','투자','우량주','전망','실적',
                 '신뢰','회복','대박','상한가','수익률',
                 '투더문','떡상','상승세','신고가','역대급',
                 '황금주','대세','상승','급등','발전',
                 '전환점','인수합병','신제품','신기술','비전',
                 '선도','전망치','잭팟','우상향','강세장','화성']

In [27]:

positive_df = pd.DataFrame({'단어': positive_words, '점수': [1.0] * len(positive_words)})

In [31]:
positive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      41 non-null     object 
 1   점수      41 non-null     float64
dtypes: float64(1), object(1)
memory usage: 788.0+ bytes


In [4]:
# 부정 단어 리스트
negative_words = ['손절','폭락','물렸다','하한가','떡락',
                  '개미털기','작전주','버블','폭탄주','축소',
                  '물타기','매도','공매도','붕괴','실패','하락',
                  '피눈물','악재','손해','물로켓','부도','리콜',
                  '변동성','퇴출','파산','잠식','위기','침체',
                  '불황','나락','폭탄','이탈','손실','냉각',
                  '마이너스','폭망','미달','부진','악화','적자',
                  '위험','불확실','불안정','혼란','실망','부정적',
                  '악성','비관적','걱정','우려','패닉','실의',
                  '고통','포기','탈출','회피','불만','퇴보',
                  '사라지다','문제'
                  ]

In [5]:
negative_df = pd.DataFrame({'단어': negative_words, '점수': [-2.0] * len(negative_words)})

In [29]:
df = pd.concat([df, positive_df], ignore_index=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425 entries, 0 to 6424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6425 non-null   object 
 1   점수      6425 non-null   float64
dtypes: float64(1), object(1)
memory usage: 100.5+ KB


In [6]:
df = pd.read_csv('positive_score_dict.csv')

In [8]:
df = pd.concat([df, negative_df], ignore_index=True)

In [12]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')

In [39]:
# 한글자 단어는 사전에서 소거
df = df[df['단어'].str.len() > 1]

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6436 entries, 0 to 6461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6436 non-null   object 
 1   점수      6436 non-null   float64
dtypes: float64(1), object(1)
memory usage: 150.8+ KB


In [41]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')